Segmenting and Clustering Neighborhoods in Toronto

import the needed libraries:

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!pip install geopy
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are missing from the target environment:
  - nodejs==12
  - scikit-learn==0.21
  - tensorflow==1.13


     |████████████████████████████████| 94 kB 2.2 MB/s eta 0:00:011
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are missing from the target environment:
  - nodejs==12
  - scikit-learn==0.21
  - tensorflow==1.13


Libraries imported.


To Scrape Data from the Toronto Wikipedia Page

In [12]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup

source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'html.parser')
table_contents=[]

table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


Shape of the DataFrame

In [6]:
df.shape

(103, 3)

To add the Latitude and Longitude values of each Postal Code

In [9]:
geo_data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")

geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


To present the DataFrame which now includes the Latitude and Longitude

In [108]:
toronto_geo_data = pd.merge(df, geo_data, on='Postal Code')

toronto_geo_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [17]:
toronto_geo_data.shape

(103, 5)

To plot each PostalCode in the Map of Toronto using Folium

In [25]:
# create map of New York using latitude and longitude values
map_toronto= folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_geo_data['Latitude'], toronto_geo_data['Longitude'], toronto_geo_data['Borough'], toronto_geo_data['Neighborhood']):
    label = '{}, {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

To get the Postal Codes in which the Borough includes the word "Toronto"

In [27]:
df_toronto_only = toronto_geo_data[toronto_geo_data['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_only.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


To check the shape of the DataFrame wherein the Borough has the word "Toronto"

In [28]:
df_toronto_only.shape

(39, 5)

The map shown below shows the Postal codes wherein the Boroughs include the word "Toronto"

In [30]:
# create map of New York using latitude and longitude values
map_toronto2= folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_only['Latitude'], df_toronto_only['Longitude'], df_toronto_only['Borough'], df_toronto_only['Neighborhood']):
    label = '{}, {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)  
    
map_toronto2

Define Foursquare Credentials and Version

In [31]:
CLIENT_ID = 'CNFGEZ4TXAUWRNSIKPRELXTHXZWVCVFF2U5UEXFO5GJCWQAP' # your Foursquare ID
CLIENT_SECRET = 'BT3IIFOWX4UL2LVGT5LSB1R1ZTKDC31OVMNH0UGJJSU3QVNQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CNFGEZ4TXAUWRNSIKPRELXTHXZWVCVFF2U5UEXFO5GJCWQAP
CLIENT_SECRET:BT3IIFOWX4UL2LVGT5LSB1R1ZTKDC31OVMNH0UGJJSU3QVNQ


To Explore the Central Bay Street Neighborhood

In [50]:
df_toronto_only.loc[5, 'Neighborhood']

'Central Bay Street'

In [51]:
neighborhood_latitude = df_toronto_only.loc[5, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto_only.loc[5, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto_only.loc[5, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Central Bay Street are 43.6579524, -79.3873826.


To obtain the nearby venues in Central Bay Street

In [64]:
radius = 1000
LIMIT = 50

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [73]:
results = requests.get(url).json()
#results

In [66]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [70]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
1,Hailed Coffee,Coffee Shop,43.658833,-79.383684
2,Tim Hortons,Coffee Shop,43.658570,-79.385123
3,NEO COFFEE BAR,Coffee Shop,43.660130,-79.385830
4,The Queen and Beaver Public House,Gastropub,43.657472,-79.383524


In [71]:
# one hot encoding
nearby_venues_onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
#nearby_venues_onehot['Neighborhood'] = nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nearby_venues_onehot.columns[-1]] + list(nearby_venues_onehot.columns[:-1])
nearby_venues_onehot = nearby_venues_onehot[fixed_columns]

nearby_venues_onehot.head()

,Yoga Studio,Art Gallery,Art Museum,Arts & Crafts Store,Bookstore,Breakfast Spot,Bubble Tea Shop,Burrito Place,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comic Shop,Cosmetics Shop,Creperie,Dessert Shop,Discount Store,Electronics Store,Fast Food Restaurant,Gastropub,Historic Site,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Neighborhood,Park,Pizza Place,Plaza,Poke Place,Sandwich Place,Shopping Mall,Sporting Goods Shop,Sushi Restaurant,Theater,University,Vegetarian / Vegan Restaurant
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


To use kMeans clustering for the nearby Venues in Central Bay Street. As a demo, thi sample uses 15 clusters

In [79]:
kclusters = 15

centralbay_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nearby_venues_onehot)


# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10]

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/srv/conda/envs/notebook/lib/python3.7/site-

Fixing the dataframe to plot

In [100]:
#nearby_venues_onehot.insert(0, 'Cluster Labels', kmeans.labels_)
#nearby_venues_onehot.insert(1,'name', nearby_venues['name'])
#nearby_venues_onehot.insert(2,'lat', nearby_venues['lat'])
#nearby_venues_onehot.insert(3,'lng', nearby_venues['lng'])
nearby_venues_onehot.head()

#nearbyvenues_centralbay_merged = nearby_venues_onehot.join(nearby_venues.set_index('categories'), on='lat')

#manhattan_merged.head() 

,Cluster Labels,name,lat,lng,Yoga Studio,Art Gallery,Art Museum,Arts & Crafts Store,Bookstore,Breakfast Spot,Bubble Tea Shop,Burrito Place,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comic Shop,Cosmetics Shop,Creperie,Dessert Shop,Discount Store,Electronics Store,Fast Food Restaurant,Gastropub,Historic Site,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Neighborhood,Park,Pizza Place,Plaza,Poke Place,Sandwich Place,Shopping Mall,Sporting Goods Shop,Sushi Restaurant,Theater,University,Vegetarian / Vegan Restaurant
0,1,Jimmy's Coffee,43.658421,-79.385613,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,Hailed Coffee,43.658833,-79.383684,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,Tim Hortons,43.658570,-79.385123,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,NEO COFFEE BAR,43.660130,-79.385830,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,The Queen and Beaver Public House,43.657472,-79.383524,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


To plot the clusters of the nearby venues of Central Bay Street in the Tornto map using Folium

In [106]:
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nearby_venues_onehot['lat'], nearby_venues_onehot['lng'], nearby_venues_onehot['name'], nearby_venues_onehot['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters